Install the Neo4j Python driver. Docs to be found here: https://neo4j.com/docs/python-manual/current/

In [ ]:
%pip install --upgrade neo4j
from neo4j import GraphDatabase, RoutingControl

In [ ]:
import pandas as pd
import numpy as np
import getpass
import os

In [ ]:
URI = input("Neo4j Database URI: ") # URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
username = 'neo4j' #replace with your username
password = getpass.getpass(prompt='Neo4j password: ')
AUTH = (username, password)
database = 'neo4j' #replace with your database name
root_dir = 'source' #replace with your root directory
file_name = 'sample.csv' #replace with your file name
rows_per_batch = 100000 #specify your desired batch size

Connect to the database

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

Load csv to pandas dataframes

In [ ]:
df = pd.DataFrame()
df = pd.read_csv(os.path.join(root_dir, file_name), header=0, sep=',', encoding='utf-8', converters={'id':int, 'col1':float, 'col2':float, 'col3':float})

Check the shape and content

In [ ]:
print(df.shape)
df.head()

Load data to the graph in batches

In [ ]:
chunks = df.shape[0]/rows_per_batch

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for chunk in np.array_split(df, chunks.__ceil__()):
        summary = driver.execute_query("""
                    UNWIND range(0, $len - 1) AS i 
                    CREATE (s:Label1{
                        nodeId: $data['id'][i],
                        prop1: $data['col1'][i],
                        prop2: $data['col2'][i],
                        prop3: $data['col3'][i]})
                    CREATE (t:Label2{
                        nodeId: $data['id'][i],
                        prop1: $data['col1'][i],
                        prop2: $data['col2'][i],
                        prop3: $data['col3'][i]})
                    CREATE (s)-[:RELATIONSHIP]->(t)
                    """,
                    data=chunk,
                    len=len(chunk),
                    database_=database,
                    routing_= RoutingControl.WRITE
                ).summary
        print("Created {nodes_created} nodes and {relationships_created} relationships and set {property_updates} properties in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        relationships_created=summary.counters.relationships_created,
        property_updates=summary.counters.properties_set,
        time=summary.result_available_after
        ))